In [1]:
import numpy as np
from astropy.table import Table, join, vstack, hstack

In [108]:
zhao1 = Table.read('Zhao_2016ApJ...833..225Z_VizieR.fits',1)
zhao1['Name_x'] = [name.replace(' ','') for name in zhao1['Name']]
zhao1.remove_columns(['Teff','logg','__Fe_H_','xi'])
zhao2 = Table.read('Zhao_2016ApJ...833..225Z_VizieR.fits',2)
sitnova = Table.read('Sitnova_2015ApJ...808..148S.csv')
sitnova.rename_column('Name','Name_x')

In [109]:
zhao1_sitnova = join(zhao1, sitnova, keys = 'Name_x', join_type='left')
zhao1_sitnova.sort('recno')

In [110]:
merged = Table()

for key in zhao1_sitnova.keys():
    merged[key] = zhao1_sitnova[key]
    
for species in np.unique(zhao2['Species']):
    
    if species != 'CH   ':

        if species[-2:] == ' I':
            element = species[:-2].replace(" ", "")
        else:
            element = species[:-3].replace(" ", "")

        merged[element+'_H'] = np.zeros(len(merged['Name']))
        merged['e_'+element+'_H'] = np.zeros(len(merged['Name']))
        merged['flag_'+element+'_H'] = np.zeros(len(merged['Name']),dtype=np.int32)
        merged[element+'_H_LTE'] = np.zeros(len(merged['Name']))
        merged['e_'+element+'_H_LTE'] = np.zeros(len(merged['Name']))
        merged['flag_'+element+'_H_LTE'] = np.zeros(len(merged['Name']),dtype=np.int32)

        for star_index, star in enumerate(merged['Name']):

            this_star_and_element = (
                (zhao2['Name'] == star) & 
                (zhao2['Species'] == species)
            )
            if len(zhao2['Species'][this_star_and_element]) == 1:
                merged[element+'_H'][star_index] = np.array(zhao2[this_star_and_element]['__X_H_NLTE'],dtype=np.float32)
                merged['e_'+element+'_H'][star_index] = np.array(zhao2[this_star_and_element]['e__X_H_NLTE'],dtype=np.float32)                
                merged[element+'_H_LTE'][star_index] = np.array(zhao2[this_star_and_element]['__X_H_LTE'],dtype=np.float32)
                merged['e_'+element+'_H_LTE'][star_index] = np.array(zhao2[this_star_and_element]['e__X_H_LTE'],dtype=np.float32)
            else:
                raise ValueError('No entry found for '+star+' and '+species)
                
        no_detection = np.isnan(merged[element+'_H'])
        merged['flag_'+element+'_H'][no_detection] += 2
        merged['flag_'+element+'_H_LTE'][no_detection] += 2

        upper_limit = np.isfinite(merged[element+'_H']) & np.isnan(merged['e_'+element+'_H'])
        merged['flag_'+element+'_H'][upper_limit] += 1
        merged['flag_'+element+'_H_LTE'][upper_limit] += 1
        
merged.write('Zhao_2016ApJ...833..225Z_VizieR_merged.fits',overwrite=True)